In [1]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, TaskType, get_peft_model

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, TaskType, get_peft_model

import torch
from sklearn.metrics import accuracy_score
from transformers import DataCollatorWithPadding

from datasets import load_from_disk, concatenate_datasets, Dataset
from data_utils import compile_DatasetFrames

/home/siebenschuh/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preliminary Analysis:
- $n \approx 15,000$ (slightly low)
- $N_{str} \leq 1,600$ (low-ish maximum number of characters considered)
- embeddings (if used) from `gist` (moderately sized model)

### Load dataset

In [2]:
# load frames
df_train, df_test, df_val = compile_DatasetFrames()

# convert to datasets
dset_train = Dataset.from_pandas(df_train)
#dset_test = Dataset.from_pandas(df_test) # ignore test for a bit
dset_val = Dataset.from_pandas(df_val)

/home/siebenschuh/Projects/dataprep/code/DPO/data_utils.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.loc[:, 'journal_cls'] = list(df_all.copy()['path'].str.split('/').str[-3].map(mapping))


Train-Val Overlap: 0
Train-Test Overlap: 0
Val-Test Overlap: 0
df_train, df_test, df_val


### Model: `peft`

In [ ]:
# Config
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

In [ ]:
# load model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [3]:
# Step 1: Load the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

# Step 2: Tokenize the dataset
dset_train_encoded = dset_train.map(preprocess_function, batched=True)
dset_val_encoded = dset_val.map(preprocess_function, batched=True)

# Remove unnecessary columns
dset_train_encoded = dset_train_encoded.remove_columns(['text', 'embeddings', 'path', 'abstract', 'firstpage', '__index_level_0__'])
dset_val_encoded = dset_val_encoded.remove_columns(['text', 'embeddings', 'path', 'abstract', 'firstpage', '__index_level_0__'])

# Set the label column
dset_train_encoded = dset_train_encoded.rename_column('journal_cls', 'labels')
dset_val_encoded = dset_val_encoded.rename_column('journal_cls', 'labels')

/eagle/projects/tpc/siebenschuh/envs_/dpo/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 2290/2290 [00:00<00:00, 7943.42 examples/s]


In [5]:
# Set format for PyTorch
dset_train_encoded.set_format("torch")
dset_val_encoded.set_format("torch")

# Step 3: Load the model
num_labels = len(set(dset_train['journal_cls']))  # Number of unique classes
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Step 4: Set up LoRA config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
    inference_mode=False,
    r=8,  # The rank of the low-rank matrices
    lora_alpha=32,
    lora_dropout=0.1
)

# Step 5: Apply LoRA to the base model using PEFT
model = get_peft_model(base_model, lora_config)

# Data collator (for dynamic padding)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 6: Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

# Step 7: Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Set to "epoch"
    save_strategy="epoch",  # Match with evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,  # Load the best model
)

# Step 8: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dset_train_encoded,
    eval_dataset=dset_val_encoded,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Step 9: Train the model with LoRA applied
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/eagle/projects/tpc/siebenschuh/envs_/dpo/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_f

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
